The following example is meant to illustrate basic usage of `evopython`. To begin, the package should be installed and this notebook run in the same environment. In addition, [human gene annotation data](https://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.chr.gtf.gz) and the 10 primates multiple whole-genome alignment must be downloaded from *Ensembl* (because we work with just the human transcription factor HES3 below, it's only necessary to download the [initial chromosome 1 MAF file](https://ftp.ensembl.org/pub/release-109/maf/ensembl-compara/multiple_alignments/10_primates.epo/10_primates.epo.1_1.maf.gz)).

`evopython` expects MAF files to be in a directory and be named after the chromosome that they're indexed on, so `1_1.maf` should be placed in a directory—which, below, I'll assume is named `10_primates` and in the current working directory—and renamed `1.maf`.

After the required data has been downloaded and organized, we can start using `evopython`:

In [14]:
from evopython import GTF, MAF

genes = GTF("Homo_sapiens.GRCh38.109.chr.gtf")

`genes` is nested dictionary-like data structure with gene name keys, so we can selectively access some gene by name or iterate over the entire gene set. Let's grab the transcription factor HES3:

In [15]:
HES3 = genes['HES3']

The high-level gene dictionaries have two keys by default, `attr` and `feat`; the former stores annotated attributes in a dictionary,

In [16]:
HES3_attr = HES3['attr']
HES3_attr

{'gene_id': 'ENSG00000173673',
 'gene_version': 9,
 'gene_name': 'HES3',
 'gene_source': 'ensembl_havana',
 'gene_biotype': 'protein_coding'}

whereas the latter stores the gene's `Feature` instance,

In [17]:
HES3_feat = HES3['feat']
HES3_feat

Feature(chrom='1', start=6244178, end=6245578, strand='+')

We gather the `Feature` instance from `genes` here, but it's worth noting that `Feature` instances are simply initialized with 0-based positional information, i.e.,

In [18]:
from evopython import Feature

HES3_feat == Feature(chrom="1", start=6244178, end=6245578, strand="+")

True

It's these instances of the `Feature` class that are resolved from the whole-genome alignment, using the `MAF` class's `get` method:

In [19]:
wga = MAF("10_primates", aligned_on="homo_sapiens")
alignments = wga.get(HES3_feat)

`aligned_on` specifies the name of the species the files are indexed on, using the same name that's used to designate the species in the MAF file, e.g., the `1` in `1.maf` is the human chromosome 1, which is designated `homo_sapiens.1` in each alignment record. `alignments` is a list because, in some cases, the feature does not have a contiguous alignment: it might be split over more than one chromosome in another species' genome, or some intervening region may have failed to align.

If the feature is contiguous, as it is in this case, we can unpack `alignments` like so:

In [20]:
alignment, = alignments

`alignment` is a dictionary mapping species to `tuple`, where `tuple[0]` is a `Feature` instance describing the alignment's position in the associated species' genome and `tuple[1]` the aligned sequence:

In [21]:
for species, (feat, seq) in alignment.items():
    name = f"{species}={feat.locus(base=1, strand=True)}"
    print("{: <50}".format(name), seq[:10])  # Just the first ten bases to save some room.

homo_sapiens=1:6244179-6245578:+                   ATGGACTGCA
pan_paniscus=1:6265551-6267008:+                   ATGGACTGCA
pan_troglodytes=1:5501313-5502746:+                ATGGACTGCA
gorilla_gorilla=1:5955740-5957141:+                ATGGACTGCA
pongo_abelii=1:222410362-222411774:-               ATGGACTGCA
nomascus_leucogenys=24:6251749-6252870:+           ATGAACTGCA
chlorocebus_sabaeus=20:125494692-125496061:-       ACGGACTGCA
macaca_fascicularis=1:218253904-218255170:-        ACGGACTGCA
macaca_mulatta=1:218281706-218283027:-             ACGGACTGCA
microcebus_murinus=2:65610888-65612183:-           ATAAAGAGCA


`Feature` instances have convenient helper methods, such as `locus()`, which enables us to construct a neat string in generic genome browser format. `base=1` indicates a 1-based, fully-closed coordinate system and `strand=True` indicates that the strand should be included at the end of the string.

Another method implemented in the `Feature` class is `pad()`, which takes three arguments:
- `pad5`, the number of bases to add onto the 5'-end of the feature;
- `pad3`, the number of bases to add onto the 3'-end of the feature; and
- `center`, an integer—5, 3, or 0—indicating whether the padding should centered on the 5' coordinate, the 3' coordinate, or neither, in which case `pad5` is applied to the 5' coordinate and `pad3` to the 3' coordinate.

I use "add" to describe feature extension, but of course, the actual operation is strand- and coordinate-dependent, e.g.,

In [22]:
forward_feat = Feature("1", 100, 200, "+")
reverse_feat = Feature("1", 100, 200, "-")

for center in (0, 5, 3):
    print(f"center={center}")

    for pad5 in (50, 100):
        for pad3 in (25, 75):
            print(f"> pad5={pad5}, pad3={pad3}")

            padded_forward_feat = forward_feat.pad(center=center, pad5=pad5, pad3=pad3)
            padded_reverse_feat = reverse_feat.pad(center=center, pad5=pad5, pad3=pad3)

            print(f"\tforward_feat={padded_forward_feat.locus(base=0)}")
            print(f"\treverse_feat={padded_reverse_feat.locus(base=0)}")

center=0
> pad5=50, pad3=25
	forward_feat=1:50-225
	reverse_feat=1:75-250
> pad5=50, pad3=75
	forward_feat=1:50-275
	reverse_feat=1:25-250
> pad5=100, pad3=25
	forward_feat=1:0-225
	reverse_feat=1:75-300
> pad5=100, pad3=75
	forward_feat=1:0-275
	reverse_feat=1:25-300
center=5
> pad5=50, pad3=25
	forward_feat=1:50-125
	reverse_feat=1:175-250
> pad5=50, pad3=75
	forward_feat=1:50-175
	reverse_feat=1:125-250
> pad5=100, pad3=25
	forward_feat=1:0-125
	reverse_feat=1:175-300
> pad5=100, pad3=75
	forward_feat=1:0-175
	reverse_feat=1:125-300
center=3
> pad5=50, pad3=25
	forward_feat=1:150-225
	reverse_feat=1:75-150
> pad5=50, pad3=75
	forward_feat=1:150-275
	reverse_feat=1:25-150
> pad5=100, pad3=25
	forward_feat=1:100-225
	reverse_feat=1:75-200
> pad5=100, pad3=75
	forward_feat=1:100-275
	reverse_feat=1:25-200


`pad()` can be useful for the multitude of regulatory features that have contigent boundaries. Returning to the HES3 transcription factor, we could derive a core promoter region by selecting the 50 bases upstream from the canonical transcription start site:

In [23]:
HES3_prom = HES3_feat.pad(center=5, pad5=50, pad3=0)

And because `pad()` just returns a new, padded feature, we can pass it to `MAF.get()` for resolution:

In [24]:
prom_alignments = wga.get(HES3_prom)

if len(prom_alignments) == 1:
    alignment, = prom_alignments
    for species, (feat, seq) in alignment.items():
        name = f"{species}={feat.locus(base=1, strand=True)}"
        print("{: <50}".format(name), seq)

homo_sapiens=1:6244129-6244178:+                   GACATGTAAACGAGGGT-CCCTATAAAGGCGGCGCTGCCTCGC-ACTGGTGC
pan_paniscus=1:6265501-6265550:+                   GACATGTAAACGAGGGT-CCCTATAAAGGCGGCGCTGCCTCGC-ACTGGTGC
pan_troglodytes=1:5501263-5501312:+                GACATGTAAACGAGGGT-CCCTATAAAGGCGGCGCTGCCTCGC-ACTGGTGC
gorilla_gorilla=1:5955690-5955739:+                GACATGTAAACAAGGGT-CCCTATAAAGGCGGCGCTGCCTCGC-ACTGGTGC
pongo_abelii=1:222411775-222411824:-               GACATGTAAACGAGGGT-CCCTATAAAGGCGGCGCTGCCTCGC-ACTGGTGC
nomascus_leucogenys=24:6251699-6251748:+           GACATGTAAACGAGGGT-CCCTATAAAGGCGGCGCTGCCTCGC-ACTGGCAC
chlorocebus_sabaeus=20:125496062-125496111:-       GACATGTAAACGAGGGT-CCCTATAAAGGCAGCACTGCCTCAC-ACTGGTGC
macaca_fascicularis=1:218255171-218255220:-        GACATGTAAACGAGGGT-CCCTATAAAGGCAGCACTGCCTCAC-ACTGGTGC
macaca_mulatta=1:218283028-218283077:-             GACATGTAAACGAGGGT-CCCTATAAAGGCAGCACTGCCTCAC-ACTGGTGC
microcebus_murinus=2:65612184-65612235:-           GACATGTAAACGA

Here, we focused on a single gene; see the `human_mouse.ipynb` notebook for how `evopython` can be used to perform genome-scale analyses.